In [6]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit,train_test_split,cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,mean_squared_error

In [ ]:
#Imports the titanic dataset
df=pd.read_csv('/kaggle/input/titanic-dataset/Titanic-Dataset.csv')
#Dropping NaN values in embarked cause they are only few unlike cabin
df=df.dropna(subset=['Embarked'])
df_labels=df['Survived']
df_cat=df[['Sex','Pclass','Cabin','Embarked']].copy()
df_num=df.drop(['Sex','Pclass','Name','Cabin','Embarked'],axis=1)
#Instead of filling in cabin values, converting it into a boolean category 
df_cat['HasCabin']=df_cat['Cabin'].notna().astype(int)
#Dropping Cabin because it is no longer useful.It has been replaced by HasCabin.
df_cat=df_cat.drop(['Cabin'],axis=1)
#People travelling together had same ticket number. Therefore converting ticket numbers into family size is more intuitive
df_num['Family Size']=df_num['Ticket'].map(df_num['Ticket'].value_counts())
#Dropping Tickets because its obsolete.
df_num=df_num.drop(['Ticket','Survived'],axis=1)
#Adding Has Cabin to the original DataFrame as well. Same with family size
df['HasCabin']=df['Cabin'].notna().astype(int)
df['Family Size']=df_num['Family Size']


In [ ]:
#Training another machine learning model to predict Age 
df_age=df.dropna(subset=['Age'])
age_df_num=df_num.dropna(subset=['Age']).drop(['Age'],axis=1)#dropped Na values of age first to match df_age(in terms of the no of rows)
df_missing=df[df['Age'].isna()]
df_lables_age=df_age['Age']
df_missing.head()

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,HasCabin,Family Size
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0,1
17,18,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,244373,13.0000,NaN,S,0,1
19,20,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,2649,7.2250,NaN,C,0,1
26,27,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,2631,7.2250,NaN,C,0,1
28,29,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,330959,7.8792,NaN,Q,0,1


In [ ]:
cat_encoder=OneHotEncoder()

In [ ]:
df_num_pipe=Pipeline([
    ('imputer',SimpleImputer(strategy='median')),
    ('std_scaler', StandardScaler())
])

In [ ]:
#Final Pipeline for cleaning the titanic dataset without the age parameter
num_attrbs=list(age_df_num) 
cat_attrbs=list(df_cat) 
final_pipeline_age=ColumnTransformer([
    ('num',df_num_pipe,num_attrbs), 
    ('cat',cat_encoder,cat_attrbs)]) 
df_final_age=final_pipeline_age.fit_transform(df_age)

In [ ]:
#Final Pipeline for cleaning the titanic dataset
# Function to handle both versions of the dataset: with or without Age
def pipeline(df_num,df_cat,df,age=True):
    num_attrbs=list(df_num) 
    if age:
        num_attrbs=['Age']+list(df_num)
    cat_attrbs=list(df_cat) 
    final_pipeline=ColumnTransformer([
        ('num',df_num_pipe,num_attrbs), 
        ('cat',cat_encoder,cat_attrbs)]) 
    return final_pipeline.fit_transform(df)

In [ ]:
df_final=pipeline(df_num,df_cat,df,False)

In [ ]:
#train_test_split for the regression model
x_train_age, x_test_age, y_train_age, y_test_age = train_test_split(
    df_final_age, 
    df_lables_age, 
    test_size=0.2, 
    random_state=42
)

In [ ]:
#Splits the dataset for training and testing purposes
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(df_final, df_labels):
    x_train, x_test = df_final[train_index], df_final[test_index]
    y_train, y_test = df_labels.iloc[train_index], df_labels.iloc[test_index]

In [ ]:
for_reg=RandomForestRegressor()
for_reg.fit(x_train_age,y_train_age)

RandomForestRegressor()

In [ ]:
predictions_age=for_reg.predict(x_test_age)
mse=mean_squared_error(y_test_age,predictions_age)
rmse=np.sqrt(mse)
cvs=cross_val_score(for_reg,x_test_age,y_test_age,scoring='neg_mean_squared_error',cv=10)
rmse_scores=np.sqrt(-cvs)
print(f'''
rmse:{rmse}
scoring:{rmse_scores}''')


rmse:13.679744758982498
scoring:[14.94376124 16.24192445 16.23095222 12.61905638 18.36405831 14.3629373
 14.38789149 18.98763084 12.98546069 14.28016069]


In [ ]:
gr_reg=GradientBoostingRegressor()
gr_reg.fit(x_train_age,y_train_age)

GradientBoostingRegressor()

In [ ]:
predictions_age=gr_reg.predict(x_test_age)
mse=mean_squared_error(y_test_age,predictions_age)
rmse=np.sqrt(mse)
cvs=cross_val_score(gr_reg,x_test_age,y_test_age,scoring='neg_mean_squared_error',cv=10)
rmse_scores=np.sqrt(-cvs)
print(f'''
rmse:{rmse}
scoring:{rmse_scores}''')


rmse:13.614491360533881
scoring:[16.09004753 16.96802859 16.90309807 13.4398546  18.88734987 18.20417232
 12.01645945 20.3589044  15.45844769 15.352321  ]


In [ ]:
lin_reg=LinearRegression()
lin_reg.fit(x_train_age,y_train_age)

LinearRegression()

In [ ]:
predictions_age=lin_reg.predict(x_test_age)
mse=mean_squared_error(y_test_age,predictions_age)
rmse=np.sqrt(mse)
cvs=cross_val_score(lin_reg,x_test_age,y_test_age,scoring='neg_mean_squared_error',cv=10)
rmse_scores=np.sqrt(-cvs)
print(f'''
rmse:{rmse}
scoring:{rmse_scores}''')


rmse:14.194158192929594
scoring:[12.32039586 15.94267612 16.37834888 14.10335964 16.18628407 14.41953773
 13.19263706 17.21765441 12.86486702 14.46615734]


In [ ]:
for_class=RandomForestClassifier()
for_class.fit(x_train,y_train)

RandomForestClassifier()

In [ ]:
df_predictions=for_class.predict(x_test)
accuracy=accuracy_score(y_test,df_predictions)
cm=confusion_matrix(y_test,df_predictions)
report=classification_report(y_test,df_predictions)
print(f'''Accuracy:{accuracy}
Confusion Matrix:
{cm}
Classification Report:
{report}''')

Accuracy:0.7921348314606742
Confusion Matrix:
[[93 17]
 [20 48]]
Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.85      0.83       110
           1       0.74      0.71      0.72        68

    accuracy                           0.79       178
   macro avg       0.78      0.78      0.78       178
weighted avg       0.79      0.79      0.79       178



In [ ]:
gr=GradientBoostingClassifier()
gr.fit(x_train,y_train)

GradientBoostingClassifier()

In [ ]:
df_predictions_1=gr.predict(x_test)
accuracy=accuracy_score(y_test,df_predictions_1)
cm=confusion_matrix(y_test,df_predictions_1)
report=classification_report(y_test,df_predictions_1)
print(f'''Accuracy:{accuracy}
Confusion Matrix:
{cm}
Classification Report:
{report}''')

Accuracy:0.8258426966292135
Confusion Matrix:
[[96 14]
 [17 51]]
Classification Report:
              precision    recall  f1-score   support

           0       0.85      0.87      0.86       110
           1       0.78      0.75      0.77        68

    accuracy                           0.83       178
   macro avg       0.82      0.81      0.81       178
weighted avg       0.82      0.83      0.83       178



In [ ]:
df_final_missing_age=final_pipeline_age.transform(df_missing)
predicted_ages_for_titanic=for_reg.predict(df_final_missing_age).flatten()
df.loc[df_missing.index, 'Age'] = predicted_ages_for_titanic

In [ ]:
df_final_predicted=pipeline(df_num,df_cat,df)

In [ ]:
split=StratifiedShuffleSplit(n_splits=1,test_size=0.2,random_state=42)
for train_index, test_index in split.split(df_final_predicted, df_labels):
    x_train_1, x_test_1 = df_final_predicted[train_index], df_final_predicted[test_index]
    y_train_1, y_test_1 = df_labels.iloc[train_index], df_labels.iloc[test_index]

In [ ]:
gr_age=GradientBoostingClassifier()
gr_age.fit(x_train_1,y_train_1)

GradientBoostingClassifier()

In [ ]:
df_predictions_1_age=gr_age.predict(x_test_1)
accuracy=accuracy_score(y_test_1,df_predictions_1_age)
cm=confusion_matrix(y_test_1,df_predictions_1_age)
report=classification_report(y_test_1,df_predictions_1_age)
print(f'''Accuracy:{accuracy}
Confusion Matrix:
{cm}
Classification Report:
{report}''')

Accuracy:0.8146067415730337
Confusion Matrix:
[[96 14]
 [19 49]]
Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.87      0.85       110
           1       0.78      0.72      0.75        68

    accuracy                           0.81       178
   macro avg       0.81      0.80      0.80       178
weighted avg       0.81      0.81      0.81       178

